In [1]:
import time
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from prettytable import PrettyTable
from tqdm import tqdm
from torchvision.models import squeezenet1_1, mobilenet_v3_small, mobilenet_v3_large, resnet50
from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v3_small
from torchvision.transforms import v2
from tools.data import data_generator
from tools.fit import train, validate, test

# Preprocessing and Setup

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 200
num_epochs = 7

# TODO: add more transforms - RandomPerspective/ RandomRotation/ FiveCrop OR RandomCrop &OR RandomResizedCrop
train_transform = v2.Compose([v2.ToImage(), 
                              v2.ToDtype(torch.float32, scale=True)
                              #v2.RandomPerspective(),
                              #v2.RandomRotation(degrees=30),
                              #v2.FiveCrop(size=(48,48)),
                              #v2.Normalize()
                              ])

eval_transform = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])

In [3]:
# --- LOAD DATA ---
train_data, val_data, test_data = data_generator(train_transform, 
                                                 eval_transform,
                                                 num_classes)

train_dl = DataLoader(train_data, batch_size=64,
                      shuffle=True, num_workers=4,
                      pin_memory=True)
val_dl = DataLoader(val_data, batch_size=64,
                    shuffle=True, num_workers=4,
                    pin_memory=True)
test_dl = DataLoader(test_data, batch_size=64,
                     shuffle=True, num_workers=4,
                     pin_memory=True)

# Training

In [4]:
squeezenet_model = squeezenet1_1(weights='DEFAULT')
# redefine networks final classifier
squeezenet_model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=1)
squeezenet_model.num_classes = num_classes
# send to gpu
squeezenet_model = squeezenet_model.to(device)

squeeze_loss_fn = nn.CrossEntropyLoss()
squeeze_optimizer = torch.optim.Adam(params=squeezenet_model.parameters(), lr = 1e-5)

# initialize training/val loss & accuracy
squeeze_train_loss = []
squeeze_train_acc = []

for epoch in range(num_epochs):
    # training loop
    train_epoch_loss, train_epoch_acc = train(squeezenet_model, device, train_dl, squeeze_optimizer, squeeze_loss_fn)
    squeeze_train_loss.append(train_epoch_loss)
    squeeze_train_acc.append(train_epoch_acc)
    print(train_epoch_loss, train_epoch_acc)
    
torch.save(squeezenet_model.state_dict(), 'squeezenet1_1_state.pt')

4.0808116114854815 tensor(13.2811, device='cuda:0')
2.620985173690319 tensor(32.1272, device='cuda:0')
2.1120528603196145 tensor(41.7580, device='cuda:0')
1.8275185229063033 tensor(47.8981, device='cuda:0')
1.6374053723216058 tensor(52.2317, device='cuda:0')
1.4997751867651938 tensor(55.5587, device='cuda:0')
1.392451539260149 tensor(58.1664, device='cuda:0')


In [5]:
# --- INSTANTIATE / LOAD MODEL ---
mobilenet_model = mobilenet_v3_small(weights='DEFAULT')
# get number of in features from source
num_features = mobilenet_model.classifier[3].in_features
# redefine the networks final fully connected layer
mobilenet_model.classifier[3] = nn.Linear(num_features, num_classes)
# send to gpu
mobilenet_model = mobilenet_model.to(device)

# --- TRAINING ---
mobile_loss_fn = nn.CrossEntropyLoss()
mobile_optimizer = torch.optim.Adam(params=mobilenet_model.parameters(), lr=1e-4)

# initialize training/val loss & accuracy
mobile_train_loss = []
mobile_train_acc = []
# mobile_val_loss = []
# mobile_val_acc = []

# table = PrettyTable()
# table.field_names = ['Epoch', 'Training Loss', 'Validation Loss', 'Training Accuracy', 'Validation Accuracy']

for epoch in range(num_epochs):
    # training loop
    train_epoch_loss, train_epoch_acc = train(mobilenet_model, device, train_dl, mobile_optimizer, mobile_loss_fn)
    mobile_train_loss.append(train_epoch_loss)
    mobile_train_acc.append(train_epoch_acc)
    print(train_epoch_loss, train_epoch_acc)
#     # validation loop
#     val_epoch_loss, val_epoch_acc = validate(mobilenet_model, device, val_dl, loss_fn)
#     mobile_val_loss.append(val_epoch_loss)
#     mobile_val_acc.append(val_epoch_acc)
#     # store epoch results in prettytable row
# TODO: Convert epoch stats from Tensor to int or something roundable
#     row = [
#         round(epoch+1, 4),  # epoch is an integer, so no need to convert
#         round(train_epoch_loss.item(), 4) if isinstance(train_epoch_loss, torch.Tensor) else round(train_epoch_loss, 4),
#         round(val_epoch_loss.item(), 4) if isinstance(val_epoch_loss, torch.Tensor) else round(val_epoch_loss, 4),
#         round(train_epoch_acc.item(), 4) if isinstance(train_epoch_acc, torch.Tensor) else round(train_epoch_acc, 4),
#         round(val_epoch_acc.item(), 4) if isinstance(val_epoch_acc, torch.Tensor) else round(val_epoch_acc, 4)
#     ]
#     table.add_rows([row])
# print(table)

# # --- RESULTS VISUALIZATION ---
# fig, ax = plt.subplots(1, 1, figsize=(16, 5))  

# ax.plot(mobile_train_loss, label='Train Loss', color='red')
# ax.plot(mobile_val_loss, label='Validation Loss', color='blue')

# ax.set_title('MobileNetV3(Small) Loss vs Epochs')
# ax.set_xlabel('Epochs')
# ax.set_ylabel('Loss')
# ax.legend()
# ax.grid(True)

torch.save(mobilenet_model.state_dict(), 'mobilenetV3_small_state.pt')

1.7105273019492626 tensor(52.6189, device='cuda:0')
0.861611755746603 tensor(71.9130, device='cuda:0')
0.692253957670927 tensor(76.6825, device='cuda:0')
0.6026886909201741 tensor(79.3466, device='cuda:0')
0.5404472106173634 tensor(81.3052, device='cuda:0')
0.49160305255800485 tensor(82.9512, device='cuda:0')
0.4558044115826487 tensor(84.0566, device='cuda:0')


In [6]:
# loading pretrained model from save state

# mobile_loss_fn = nn.CrossEntropyLoss()
# # --- INSTANTIATE / LOAD MODEL ---
# mobilenet_model = mobilenet_v3_small(weights='DEFAULT')
# # get number of in features from source
# num_features = mobilenet_model.classifier[3].in_features
# # redefine the networks final fully connected layer
# mobilenet_model.classifier[3] = nn.Linear(num_features, num_classes)
# # send to gpu
# mobilenet_model = mobilenet_model.to(device)
# mobilenet_model.load_state_dict(torch.load('mobilenetV3_small_state.pt'))

In [7]:
# --- INSTANTIATE / LOAD MODEL ---
mobilenet_l_model = mobilenet_v3_large(weights='DEFAULT')
# get number of in features from source
num_features = mobilenet_l_model.classifier[3].in_features
# redefine the networks final fully connected layer
mobilenet_l_model.classifier[3] = nn.Linear(num_features, 200)
# send to gpu
mobilenet_l_model = mobilenet_l_model.to(device)

# --- TRAINING ---
mobile_l_loss_fn = nn.CrossEntropyLoss()
mobile_l_optimizer = torch.optim.Adam(params=mobilenet_l_model.parameters(), lr=1e-4)

# initialize training/val loss & accuracy
mobile_l_train_loss = []
mobile_l_train_acc = []
# mobile_l_val_loss = []
# mobile_l_val_acc = []

# table = PrettyTable()
# table.field_names = ['Epoch', 'Training Loss', 'Validation Loss', 'Training Accuracy', 'Validation Accuracy']

for epoch in range(num_epochs):
    # training loop
    train_epoch_loss, train_epoch_acc = train(mobilenet_l_model, device, train_dl, mobile_l_optimizer, mobile_l_loss_fn)
    mobile_l_train_loss.append(train_epoch_loss)
    mobile_l_train_acc.append(train_epoch_acc)
    print(train_epoch_loss, train_epoch_acc)

torch.save(mobilenet_l_model.state_dict(), 'mobilenetV3_large_state.pt')

1.1234707615152002 tensor(67.0748, device='cuda:0')
0.4851280972495675 tensor(83.0944, device='cuda:0')
0.3715459684550762 tensor(86.6842, device='cuda:0')
0.30347563659250737 tensor(89.1323, device='cuda:0')
0.25895253277905284 tensor(90.5831, device='cuda:0')
0.22678376735411584 tensor(91.6781, device='cuda:0')
0.2012403815136291 tensor(92.5180, device='cuda:0')


In [8]:
resnet_model = resnet50(weights='DEFAULT')
# get number of in_features from source
num_features = resnet_model.fc.in_features
# redefine the networks final fully connected layer
resnet_model.fc = nn.Linear(num_features, num_classes)
# send to gpu
resnet_model = resnet_model.to(device)


res_loss_fn = nn.CrossEntropyLoss()
res_optimizer = torch.optim.Adam(params=resnet_model.parameters(), lr = 1e-5)

# initialize training loss
res_train_loss = []
res_train_acc = []

for epoch in range(num_epochs):
    # training loop
    train_epoch_loss, train_epoch_acc = train(mobilenet_model, device, train_dl, mobile_optimizer, mobile_loss_fn)
    mobile_train_loss.append(train_epoch_loss)
    mobile_train_acc.append(train_epoch_acc)
    print(train_epoch_loss, train_epoch_acc)
    
torch.save(resnet_model.state_dict(), 'resnet50_state.pt')

0.4253082916975021 tensor(85.0584, device='cuda:0')
0.40078090895861385 tensor(85.8206, device='cuda:0')
0.3785639877445996 tensor(86.5450, device='cuda:0')
0.3584332454048097 tensor(87.1966, device='cuda:0')
0.3426983310587704 tensor(87.6837, device='cuda:0')
0.3266990590013564 tensor(88.2011, device='cuda:0')
0.3129300588298589 tensor(88.6458, device='cuda:0')


# Results compilation and visualization

In [13]:
start_time = time.time()
squeezenet_results = ['SqueezeNet1.1'] + [round(item, 4) for item in list(test(squeezenet_model, device, test_dl, squeeze_loss_fn))]
squeezenet_results += [round(time.time() - start_time, 4)]

start_time = time.time()
mobilenet_results = ['MobileNetV3(Small)'] + [round(item, 4) for item in list(test(mobilenet_model, device, test_dl, mobile_loss_fn))]
mobilenet_results += [round(time.time() - start_time, 4)]

start_time = time.time()
mobilenet_l_results = ['MobileNetV3(Large)'] + [round(item, 4) for item in list(test(mobilenet_l_model, device, test_dl, mobile_l_loss_fn))]
mobilenet_l_results += [round(time.time() - start_time, 4)]

start_time = time.time()
resnet_results = ['ResNet50'] + [round(item, 4) for item in list(test(resnet_model, device, test_dl, res_loss_fn))]
resnet_results += [round(time.time() - start_time, 4)]

C:\Users\Quinn\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
table = PrettyTable()
table.field_names = ['Model', 'Loss', 'Accuracy', 
                     'Precision', 'Recall', 'F1 Score', 
                     'Top-3 Accuracy', 'Testing Speed (Seconds)']
table.add_rows([squeezenet_results,
                mobilenet_results,
                mobilenet_l_results,
                resnet_results])
print(table)

+--------------------+--------+----------+-----------+--------+----------+----------------+-------------------------+
|       Model        |  Loss  | Accuracy | Precision | Recall | F1 Score | Top-3 Accuracy | Testing Speed (Seconds) |
+--------------------+--------+----------+-----------+--------+----------+----------------+-------------------------+
|   SqueezeNet1.1    | 1.1708 |  0.6404  |   0.6433  | 0.6404 |  0.6375  |     0.8585     |         47.3199         |
| MobileNetV3(Small) | 0.3773 |  0.8696  |   0.8729  | 0.8696 |  0.8699  |     0.9743     |         46.0982         |
| MobileNetV3(Large) | 0.2458 |  0.9134  |   0.9154  | 0.9134 |  0.9132  |     0.9864     |         47.9899         |
|      ResNet50      | 5.4713 |  0.005   |   0.0057  | 0.005  |  0.0037  |     0.0155     |         52.4077         |
+--------------------+--------+----------+-----------+--------+----------+----------------+-------------------------+


In [12]:
# fig, ax = plt.subplots(1, 1, figsize=(16, 5))  

# ax.plot(mobile_train_loss, label='Train Loss', color='red')
# ax.plot(mobile_val_loss, label='Validation Loss', color='blue')

# ax.set_title('MobileNetV3(Small) Loss vs Epochs')
# ax.set_xlabel('Epochs')
# ax.set_ylabel('Loss')
# ax.legend()
# ax.grid(True)